In [1]:
pip install "numpy<2.0"


In [2]:
pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.7 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.


In [4]:
pip install tqdm

In [3]:
pip install seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16161 sha256=f9dac11238d8aa16a78195c5a50cfaae95fc4a958089b14b0d99e10847a2884f
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [5]:
pip install pytorch-crf


In [6]:
pip install seaborn

In [7]:
pip install nltk

In [2]:
import torch
import torch.nn as nn
from torch.nn.utils.rnn import pad_sequence
from datasets import load_dataset
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
from functools import partial
import uuid
from collections import Counter
from seqeval.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix
import numpy as np
import logging
import re


In [9]:
import logging

In [11]:
from transformers import AutoTokenizer

In [3]:
import torch
import torch.nn as nn
from torchcrf import CRF

class BiLSTMTextCNNCRF(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_labels, dropout=0.1, num_filters=100, filter_sizes=[3, 4, 5]):
        super(BiLSTMTextCNNCRF, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2, bidirectional=True, batch_first=True)

        # TextCNN components
        self.convs = nn.ModuleList([
            nn.Conv1d(hidden_dim, num_filters, kernel_size)
            for kernel_size in filter_sizes
        ])

        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(hidden_dim + len(filter_sizes) * num_filters, num_labels)
        self.crf = CRF(num_labels, batch_first=True)

    def forward(self, input_ids, attention_mask=None, labels=None):
        embedded = self.embedding(input_ids)
        lstm_out, _ = self.lstm(embedded)

        # TextCNN processing
        cnn_input = lstm_out.permute(0, 2, 1)  # Change to (batch, hidden_dim, seq_len)
        cnn_out = [torch.relu(conv(cnn_input)) for conv in self.convs]
        cnn_out = [nn.functional.max_pool1d(out, out.size(2)).squeeze(2) for out in cnn_out]
        cnn_out = torch.cat(cnn_out, dim=1)

        # Combine LSTM and CNN outputs
        combined = torch.cat([lstm_out, cnn_out.unsqueeze(1).expand(-1, lstm_out.size(1), -1)], dim=2)

        emissions = self.fc(self.dropout(combined))

        if labels is not None:
            mask = attention_mask.bool()
            loss = -self.crf(emissions, labels, mask=mask)
            return loss
        else:
            mask = attention_mask.bool()
            predictions = self.crf.decode(emissions, mask=mask)
            return predictions


In [4]:
class PIIDataset(torch.utils.data.Dataset):
    def __init__(self, examples, tokenizer, max_length):
        self.examples = examples
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        item = self.examples[idx]
        encoding = self.tokenizer(item['source_text'], truncation=True, padding='max_length', max_length=self.max_length, return_tensors='pt')
        input_ids = encoding['input_ids'].squeeze()
        attention_mask = encoding['attention_mask'].squeeze()
        label_ids = torch.tensor(item['labels'], dtype=torch.long)
        # Instead of setting padding labels to -100, keep them as "O"
        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': label_ids
        }

In [5]:
def process_data(example, labels2int, tokenizer, max_length):
    text, labels = example["source_text"], example["privacy_mask"]
    encoding = tokenizer(text, truncation=True, padding='max_length', max_length=max_length,
                         return_offsets_mapping=True, return_special_tokens_mask=True)
    label_ids = [labels2int["O"]] * len(encoding['input_ids'])
    for label in labels:
        start_char, end_char = label['start'], label['end']
        token_start = encoding.char_to_token(start_char)
        token_end = encoding.char_to_token(end_char - 1)
        if token_start is not None and token_end is not None:
            label_type = label['label']
            if label_type != "CARDISSUER":
                if label_type.startswith("GIVENNAME"):
                    label_type = "GIVENNAME"
                elif label_type.startswith("LASTNAME"):
                    label_type = "LASTNAME"

                label_ids[token_start] = labels2int.get(f'B-{label_type}', labels2int["O"])
                for i in range(token_start + 1, token_end + 1):
                    label_ids[i] = labels2int.get(f'I-{label_type}', labels2int["O"])

    # Set labels for special tokens and padding to "O" instead of -100
    for i, (input_id, special_token_mask) in enumerate(zip(encoding['input_ids'], encoding['special_tokens_mask'])):
        if special_token_mask == 1 or tokenizer.convert_ids_to_tokens(input_id).startswith("##"):
            label_ids[i] = labels2int["O"]

    return {
        'input_ids': encoding['input_ids'],
        'attention_mask': encoding['attention_mask'],
        'labels': label_ids
    }

In [6]:
from tqdm import tqdm

def train(model, train_loader, val_loader, optimizer, device, num_epochs, id2label, patience=3):
    best_val_loss = float('inf')
    epochs_no_improve = 0
    best_model = None
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}")
        for batch in progress_bar:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            loss = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            progress_bar.set_postfix({'train_loss': f"{loss.item():.4f}"})

        avg_train_loss = total_loss / len(train_loader)
        train_losses.append(avg_train_loss)
        print(f"\nEpoch {epoch+1}/{num_epochs}, Average training loss: {avg_train_loss:.4f}")

        model.eval()
        val_loss = 0
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for batch in tqdm(val_loader, desc="Validation"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                loss = model(input_ids, attention_mask=attention_mask, labels=labels)
                val_loss += loss.item()

                preds = model(input_ids, attention_mask=attention_mask)

                for i, mask in enumerate(attention_mask):
                    pred = [id2label[p] for p, m in zip(preds[i], mask) if m.item() == 1]
                    true = [id2label[l.item()] for l, m in zip(labels[i], mask) if m.item() == 1]
                    if pred and true:
                        all_preds.append(pred)
                        all_labels.append(true)

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        print(f"Validation loss: {avg_val_loss:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_no_improve = 0
            best_model = model.state_dict()
        else:
            epochs_no_improve += 1
            if epochs_no_improve == patience:
                print(f"Early stopping triggered after epoch {epoch+1}")
                model.load_state_dict(best_model)
                break

        print("\nClassification Report:")
        print(classification_report(all_labels, all_preds))

    # Plot learning curve
    plt.figure(figsize=(10, 6))
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Training Loss')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Learning Curve')
    plt.legend()
    plt.savefig('learning_curve.png')
    plt.close()

    return model, all_labels, all_preds

In [7]:
def plot_confusion_matrix(true_labels, pred_labels, label_list):
    true_flat = [item for sublist in true_labels for item in sublist]
    pred_flat = [item for sublist in pred_labels for item in sublist]

    cm = confusion_matrix(true_flat, pred_flat, labels=label_list)

    plt.figure(figsize=(20, 16))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_list, yticklabels=label_list)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.xticks(rotation=90)
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.close()

In [9]:
def main():
    # Define labels (excluding CARDISSUER)
    labels = [
        "BOD", "BUILDING", "CITY", "COUNTRY", "DATE", "DRIVERLICENSE",
        "EMAIL", "GEOCOORD", "GIVENNAME", "IDCARD", "IP", "LASTNAME",
        "PASS", "PASSPORT", "POSTCODE", "SECADDRESS", "SEX",
        "SOCIALNUMBER", "STATE", "STREET", "TEL", "TIME", "TITLE", "USERNAME"
    ]

    label_list = ["O"] + [f"B-{label}" for label in labels] + [f"I-{label}" for label in labels]
    label2id = {label: i for i, label in enumerate(label_list)}
    id2label = {v: k for k, v in label2id.items()}

    # Load tokenizer and dataset
    tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
    ds = load_dataset("ai4privacy/pii-masking-300k")
    ds = ds.filter(lambda x: x["language"] == "English", num_proc=4)
    ds = ds.filter(lambda x: "CARDISSUER" not in x["mbert_bio_labels"], num_proc=4)

    max_length = 512
    ds = ds.map(
        partial(process_data, labels2int=label2id, tokenizer=tokenizer, max_length=max_length),
        remove_columns=['privacy_mask', 'span_labels', 'mbert_text_tokens', 'mbert_bio_labels', 'id', 'language', 'set'],
        num_proc=8
    )

    # Split the validation set into validation and test sets
    val_test_split = 0.3
    val_dataset = ds["validation"]
    val_dataset = val_dataset.shuffle(seed=42)
    val_size = int(len(val_dataset) * (1 - val_test_split))
    test_size = len(val_dataset) - val_size
    val_dataset, test_dataset = val_dataset.select(range(val_size)), val_dataset.select(range(val_size, len(val_dataset)))

    # Create datasets and dataloaders
    train_dataset = PIIDataset(ds["train"], tokenizer, max_length)
    val_dataset = PIIDataset(val_dataset, tokenizer, max_length)
    test_dataset = PIIDataset(test_dataset, tokenizer, max_length)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=64)
    test_loader = DataLoader(test_dataset, batch_size=64)

    # Initialize model
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    model = BiLSTMTextCNNCRF(
        vocab_size=tokenizer.vocab_size,
        embedding_dim=300,
        hidden_dim=256,
        num_labels=len(label_list)
    ).to(device)

    # Define optimizer
    optimizer = torch.optim.Adam(model.parameters(), lr=2e-5)

    # Train the model
    num_epochs = 50
    print(f"Starting training for {num_epochs} epochs...")
    model, true_labels, pred_labels = train(model, train_loader, val_loader, optimizer, device, num_epochs, id2label)

    # Save the model
    model_save_path = f"bilstm_textcnn_crf_pii_model_{str(uuid.uuid4())}.pth"
    torch.save(model.state_dict(), model_save_path)
    print(f"Model saved successfully to {model_save_path}")

    # Test the model
    model.eval()
    test_preds = []
    test_labels = []
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            preds = model(input_ids, attention_mask=attention_mask)

            for i, mask in enumerate(attention_mask):
                pred = [id2label[p] for p, m in zip(preds[i], mask) if m.item() == 1]
                true = [id2label[l.item()] for l, m in zip(labels[i], mask) if m.item() == 1]
                if pred and true:
                    test_preds.append(pred)
                    test_labels.append(true)

    # Print test results
    print("\nTest Results:")
    print(classification_report(test_labels, test_preds))
     # After training and testing
    plot_confusion_matrix(test_labels, test_preds, label_list)


In [12]:
if __name__ == "__main__":
    main()

C:\Users\harsh\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map (num_proc=8):   0%|          | 0/29908 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/7946 [00:00<?, ? examples/s]

Using device: cuda
Starting training for 50 epochs...


Epoch 1/50: 100%|██████████████████████████████████████████████| 935/935 [06:21<00:00,  2.45it/s, train_loss=1857.7751]


...


Epoch 50/50, Average training loss: 167.8436


Validation: 100%|██████████████████████████████████████████████████████████████████████| 87/87 [01:12<00:00,  1.19it/s]


Validation loss: 459.4186

Classification Report:
               precision    recall  f1-score   support

          BOD       0.94      0.92      0.93      1916
     BUILDING       0.95      0.93      0.94      1356
         CITY       0.83      0.77      0.79      1653
      COUNTRY       0.96      0.95      0.96      1128
         DATE       0.88      0.91      0.89      1475
DRIVERLICENSE       0.89      0.85      0.87      2984
        EMAIL       0.97      0.97      0.97      5402
     GEOCOORD       0.84      0.89      0.87       305
    GIVENNAME       0.66      0.61      0.64      1854
       IDCARD       0.78      0.80      0.79      1834
           IP       0.97      0.98      0.98      6754
     LASTNAME       0.71      0.59      0.65      2281
         PASS       0.78      0.71      0.75      2270
     PASSPORT       0.84      0.86      0.85      1709
     POSTCODE       0.93      0.93      0.93      1957
   SECADDRESS       0.95      0.92      0.93       853
          SEX 

Testing: 100%|█████████████████████████████████████████████████████████████████████████| 38/38 [00:29<00:00,  1.31it/s]



Test Results:
               precision    recall  f1-score   support

          BOD       0.95      0.92      0.93       781
     BUILDING       0.95      0.89      0.92       586
         CITY       0.81      0.75      0.77       715
      COUNTRY       0.96      0.97      0.96       443
         DATE       0.88      0.89      0.88       573
DRIVERLICENSE       0.91      0.88      0.90      1298
        EMAIL       0.97      0.97      0.97      2212
     GEOCOORD       0.76      0.84      0.80       122
    GIVENNAME       0.65      0.63      0.64       743
       IDCARD       0.82      0.82      0.82       834
           IP       0.97      0.98      0.98      2856
     LASTNAME       0.75      0.61      0.67      1035
         PASS       0.84      0.77      0.81       871
     PASSPORT       0.83      0.83      0.83       752
     POSTCODE       0.91      0.94      0.92       797
   SECADDRESS       0.95      0.87      0.91       398
          SEX       0.94      0.95      0.95     